In [2]:
from pyspark.sql import SparkSession
import os

# Optional: suppress Spark INFO logs
import logging
logger = logging.getLogger('py4j')
logger.setLevel(logging.WARN)

# Ensure PySpark loads the correct Delta JAR for Spark 4.1.1
os.environ['PYSPARK_SUBMIT_ARGS'] = (
    "--packages io.delta:delta-core_2.12:3.5.0 "  # Delta 3.5.0 compatible with Spark 4.x
    "pyspark-shell"
)

# Create SparkSession with Delta support
spark = (
    SparkSession.builder
    .appName("LeadPulse_ETL")
    .master("local[*]")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

# Test Delta Lake
try:
    from delta.tables import DeltaTable
    print("✅ Delta Lake package loaded successfully")
except ImportError:
    print("❌ Delta Lake package not found")

# Example Bronze path
BRONZE_PATH = "data/bronze/delta_bronze_cdf"

try:
    bronze_df = spark.read.format("delta").load(BRONZE_PATH)
    print(f"✅ Bronze Delta table loaded from {BRONZE_PATH}")
    bronze_df.show(5)
except Exception as e:
    print("❌ Error loading Bronze Delta table:", e)


❌ Delta Lake package not found
❌ Error loading Bronze Delta table: An error occurred while calling o49.load.
: java.util.ServiceConfigurationError: org.apache.spark.sql.sources.DataSourceRegister: org.apache.spark.sql.delta.sources.DeltaDataSource Unable to get public no-arg constructor
	at java.base/java.util.ServiceLoader.fail(ServiceLoader.java:586)
	at java.base/java.util.ServiceLoader.getConstructor(ServiceLoader.java:679)
	at java.base/java.util.ServiceLoader$LazyClassPathLookupIterator.hasNextService(ServiceLoader.java:1240)
	at java.base/java.util.ServiceLoader$LazyClassPathLookupIterator.hasNext(ServiceLoader.java:1273)
	at java.base/java.util.ServiceLoader$2.hasNext(ServiceLoader.java:1309)
	at java.base/java.util.ServiceLoader$3.hasNext(ServiceLoader.java:1393)
	at scala.collection.convert.JavaCollectionWrappers$JIteratorWrapper.hasNext(JavaCollectionWrappers.scala:46)
	at scala.collection.StrictOptimizedIterableOps.filterImpl(StrictOptimizedIterableOps.scala:225)
	at scala.